In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
import pandas as pd   #to create dataframe
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML
import numpy as np  # to count the values (in our case)
# import imdb

## Scraping IMDb Top 1000 Movies Data with BeautifulSoup

In [2]:

#creating an empty list, so that we can append the values
movie_name = []
genre = []
year = []
time = []
rating = []
metascore = []
votes = []
#Note: These three list are added recently, Not explained in the video
description = []
Director = []




for i in range(1,11):
    j = str(i)
    url = 'https://www.imdb.com/list/ls048276758/?sort=user_rating,desc&st_dt=&mode=detail&page=' + j 

    response = requests.get(url)
    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-detail'})

        for store in movie_data:

            name = store.h3.a.text
            movie_name.append(name)
            try:
                type_movie = store.p.find('span', class_ = 'genre').text.replace('\n', '').replace('            ', '')
            except AttributeError:
                type_movie = None
            except ValueError:
                type_movie = None
            genre.append(type_movie)
    
            try:
                year_of_release = int(store.find('span', class_ = 'lister-item-year text-muted unbold').text[-5:-1])
            except AttributeError:
                year_of_release = None
            except ValueError:
                year_of_release = None
            year.append(year_of_release)

            runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
            time.append(runtime)

            imdb_rating = float(store.find('div', class_ = 'ipl-rating-star small').text.strip())
            rating.append(imdb_rating)

            value = store.find_all('span', attrs = {'name': 'nv'})
            vote = value[0].text
            votes.append(vote)

            meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else 'null'
            metascore.append(meta)

            describe = store.find_all('p', class_ = '')
            description_ = describe[0].text.replace('\n', '') 
            #     if len(describe) >1 else '*****'
            description.append(description_)

        movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Genre':genre, 'Year of relase': year, 'Runtime (Minutes)': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Description': description})


### Function for Retrieving Movie Data from IMDb

In [7]:
def get_movie_data(movie_name):
    try:
        ia = imdb.IMDb()
        my_list= []
        movies = ia.search_movie(movie_name)

        if not movies:
            return None

        movie = ia.get_movie(movies[0].movieID)
        ia.update(movie)        
        connections = ia.get_movie_connections(movie.movieID)
        director = movie.get('director')
        if director:
            name_director=director[0]['name']
        else:
            name_director=None
        outline = movie.get("plot outline")
        language = movie.get('languages', ['N/A'])
        crew_size = len(movie.get('cast', []))

        if 'followed by' in connections['data']['connections'] or "follows" in connections['data']['connections']:
            series= 1
        else:
            series= 0
        my_list= [series,language,crew_size, name_director,outline]        
        return my_list
    except imdb.IMDbError as e:
         return None



In [9]:
movie_DF["details"]=movie_DF["Name of movie"].apply(lambda x:get_movie_data(x))


2024-03-22 12:39:52,582 CRITICAL [imdbpy] C:\Users\tohar\AppData\Roaming\Python\Python39\site-packages\imdb\_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/find/?q=The+Godfather&s=tt', 'proxy': '', 'exception type': 'IOError', 'original exception': timeout('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "C:\Users\tohar\AppData\Roaming\Python\Python39\site-packages\imdb\parser\http\__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "c:\program files\python39\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\program files\python39\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "c:\program files\python39\lib\urllib\request.py", line 494, in _call_chain
    result = func(*args)
  File "c:\program files\python39\lib\urllib\request

## Exporting Movie Data to CSV

In [10]:
movie_DF.to_csv('movie_DF_output.csv', encoding='utf-8-sig' )